### Preparação do ambiente

In [26]:
%pip install -r requirements.txt 


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Imports, extração e geração do Bag of Words

In [1]:
import os
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Baixar recursos do NLTK (apenas na primeira execução)
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     /home/anniasebold/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anniasebold/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/anniasebold/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
# Função para extrair texto de PDFs
def pdf_para_txt(caminho_pdf):
    with open(caminho_pdf, "rb") as f:
        leitor = PyPDF2.PdfReader(f)
        texto = ""
        for pagina in range(len(leitor.pages)):
            texto += leitor.pages[pagina].extract_text()
    return texto

# Diretórios com os PDFs
diretorios = {
  'poesia': 'texts/poetry',
  'prosa':  'texts/prose',
  'jornalismo': 'texts/jornalism'
}

# Função para limpar e remover stopwords
def limpar_texto(texto):
    stop_words = set(stopwords.words("english"))
    palavras = word_tokenize(texto.lower())
    palavras_limpa = [palavra for palavra in palavras
                      if palavra.isalnum() and palavra not in stop_words]
    return " ".join(palavras_limpa)

In [3]:
# Extraindo textos e gerando classes
textos = []
classes = []

for classe, caminho in diretorios.items():
    for arquivo in os.listdir(caminho):
        if arquivo.endswith('.pdf'):
            texto = pdf_para_txt(os.path.join(caminho, arquivo))
            texto_limpo = limpar_texto(texto)
            textos.append(texto_limpo)
            classes.append(classe)

# Criando a matriz Bag of Words
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textos)

# Algoritmos de aprendizado super-visionado

Para avaliar o desempenho dos algoritmos, ser ̃ao utilizadas as seguintes métricas:

*   Acuracia: Proporção de previsões corretas em relação ao total de exemplo. Pode ser obtida com a função accuracy score.
*   F1-Score: Média harmônica da precisão e revocação, especialmnte útil para classes desbalanceadas. Pode ser calculada com f1 score.

Cada algoritmo será avaliado utilizando o procedimento de validação cruzada estratificada com 10 folds. A validação cruzada consiste em dividir os dados em 10 subconjuntos (folds), utilizando cada um como conjunto de teste e os demais como treino, em rodadas sucessivas. Esse método é implementado com a função StratifiedKFold. Para cada rodada da validação cruzada, serão coletados os seguintes valores:


*   Acurácia de cada fold;
*   F1-score de cada fold.

In [12]:
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score

# Hiperparâmetros e Otimização

Para cada algoritmo, deverão ser testadas ao menos duas combinações de hiperparâmetros.
A otimização desses parâmetros pode ser realizada utilizando o método GridSearchCV, disponível no Scikit-learn.

# KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Convertendo as classes para um array numpy
y = np.array(classes)

# Criando o GridSearchCV para buscar os melhores hiperparâmetros
grid_search = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid={
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
    },
    scoring=make_scorer(f1_score, average='weighted'),
    cv=10,  # Validação cruzada com 10 folds
    n_jobs=1,  # Paralelismo
    verbose=1   # Mostrar progresso
)

# Executando o GridSearchCV
grid_search.fit(X, y)

# Obtendo as duas melhores combinações de parâmetros
resultados = grid_search.cv_results_
melhores_indices = np.argsort(resultados['mean_test_score'])[::-1][:2]

melhores_parametros = [resultados['params'][idx] for idx in melhores_indices]

print(f"Melhores parâmetros encontrados: {melhores_parametros}")



Fitting 10 folds for each of 6 candidates, totalling 60 fits
Melhores parâmetros encontrados: [{'n_neighbors': 7, 'weights': 'distance'}, {'n_neighbors': 7, 'weights': 'uniform'}]


In [14]:
# Executando com as duas melhores combinações
for params in melhores_parametros:
  print(f"\nKNN para os hiperparâmetros: {params}\n")

  knn = KNeighborsClassifier(
      n_neighbors=params['n_neighbors'],
      weights=params['weights']
    )

  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  accuracies_fold = cross_val_score(knn, X, y, cv=kf, scoring='accuracy')
  f1_scores_fold = cross_val_score(knn, X, y, cv=kf, scoring='f1_weighted')

  for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      knn.fit(X_train, y_train)
      y_pred = knn.predict(X_test)

      # Acurácia e o F1-Score do fold atual
      print(f"Fold {fold}\nAcurácia = {accuracies_fold[fold-1]:.4f}\nF1-Score = {f1_scores_fold[fold-1]:.4f}\n")

  # Cálculo das médias e o desvio padrão
  media_acuracia = np.mean(accuracies_fold)
  desvio_acuracia = np.std(accuracies_fold)

  media_f1 = np.mean(f1_scores_fold)
  desvio_f1 = np.std(f1_scores_fold)

  print(f"Média de Acurácia: {media_acuracia:.4f}")
  print(f"Desvio Padrão de Acurácia: {desvio_acuracia:.4f}")
  print(f"Média de F1-Score: {media_f1:.4f}")
  print(f"Desvio Padrão de F1-Score: {desvio_f1:.4f}")




KNN para os hiperparâmetros: {'n_neighbors': 7, 'weights': 'distance'}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 0.9000
F1-Score = 0.8993

Fold 3
Acurácia = 0.9333
F1-Score = 0.9327

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 0.9333
F1-Score = 0.9327

Fold 6
Acurácia = 0.9000
F1-Score = 0.9019

Fold 7
Acurácia = 0.9667
F1-Score = 0.9666

Fold 8
Acurácia = 0.9333
F1-Score = 0.9327

Fold 9
Acurácia = 0.9333
F1-Score = 0.9346

Fold 10
Acurácia = 0.9333
F1-Score = 0.9332

Média de Acurácia: 0.9433
Desvio Padrão de Acurácia: 0.0335
Média de F1-Score: 0.9434
Desvio Padrão de F1-Score: 0.0334

KNN para os hiperparâmetros: {'n_neighbors': 7, 'weights': 'uniform'}

Fold 1
Acurácia = 0.9667
F1-Score = 0.9666

Fold 2
Acurácia = 0.8667
F1-Score = 0.8669

Fold 3
Acurácia = 0.9000
F1-Score = 0.8977

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 0.9333
F1-Score = 0.9327

Fold 6
Acurácia = 0.9000
F1-Score = 0.9019

Fold 7
Acurácia = 0.9333
F1-Sc

# Árvore de Decisão

In [16]:
from sklearn.tree import DecisionTreeClassifier

y = np.array(classes)

grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid={
      'criterion': ['gini', 'entropy'],  # Critérios para a divisão
      'max_depth': [None, 10, 20, 30],   # Profundidade máxima da árvore
    },
    scoring=make_scorer(f1_score, average='weighted'),
    cv=10,
    n_jobs=1,
    verbose=1
)

grid_search.fit(X, y)

resultados = grid_search.cv_results_
melhores_indices = np.argsort(resultados['mean_test_score'])[::-1][:2]

melhores_parametros = [resultados['params'][idx] for idx in melhores_indices]

print(f"Melhores parâmetros encontrados: {melhores_parametros}")


Fitting 10 folds for each of 8 candidates, totalling 80 fits
Melhores parâmetros encontrados: [{'criterion': 'entropy', 'max_depth': 30}, {'criterion': 'entropy', 'max_depth': 20}]


In [17]:
# Executando com as duas melhores combinações
for params in melhores_parametros:
  print(f"\nÁrvore de Decisão para os hiperparâmetros: {params}\n")

  dtc = DecisionTreeClassifier(
      random_state=42,
      criterion=params['criterion'],
      max_depth=params['max_depth']
    )

  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  accuracies_fold = cross_val_score(dtc, X, y, cv=kf, scoring='accuracy')
  f1_scores_fold = cross_val_score(dtc, X, y, cv=kf, scoring='f1_weighted')

  for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      dtc.fit(X_train, y_train)
      y_pred = dtc.predict(X_test)

      # Acurácia e o F1-Score do fold atual
      print(f"Fold {fold}\nAcurácia = {accuracies_fold[fold-1]:.4f}\nF1-Score = {f1_scores_fold[fold-1]:.4f}\n")

  # Cálculo das médias e o desvio padrão
  media_acuracia = np.mean(accuracies_fold)
  desvio_acuracia = np.std(accuracies_fold)

  media_f1 = np.mean(f1_scores_fold)
  desvio_f1 = np.std(f1_scores_fold)

  print(f"Média de Acurácia: {media_acuracia:.4f}")
  print(f"Desvio Padrão de Acurácia: {desvio_acuracia:.4f}")
  print(f"Média de F1-Score: {media_f1:.4f}")
  print(f"Desvio Padrão de F1-Score: {desvio_f1:.4f}")


Árvore de Decisão para os hiperparâmetros: {'criterion': 'entropy', 'max_depth': 30}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 0.9333
F1-Score = 0.9332

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 1.0000
F1-Score = 1.0000

Fold 6
Acurácia = 0.9667
F1-Score = 0.9666

Fold 7
Acurácia = 0.9000
F1-Score = 0.8997

Fold 8
Acurácia = 0.9667
F1-Score = 0.9666

Fold 9
Acurácia = 1.0000
F1-Score = 1.0000

Fold 10
Acurácia = 0.9333
F1-Score = 0.9327

Média de Acurácia: 0.9700
Desvio Padrão de Acurácia: 0.0348
Média de F1-Score: 0.9699
Desvio Padrão de F1-Score: 0.0349

Árvore de Decisão para os hiperparâmetros: {'criterion': 'entropy', 'max_depth': 20}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 0.9333
F1-Score = 0.9332

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 1.0000
F1-Score = 1.0000

Fold 6
Acurácia = 0.9667
F1-Score = 0.9666

F

# Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB

y = np.array(classes)

# Convertendo X para um array denso
X_naive = X.toarray()

grid_search = GridSearchCV(
    estimator=GaussianNB(),
    param_grid={
        'var_smoothing': [1e-9, 1e-7],  # Diferentes valores de suavização
        'priors': [None, [1/3, 1/3, 1/3]],  # Testando a distribuição de priors
    },
    scoring=make_scorer(f1_score, average='weighted'),
    cv=10,
    n_jobs=1,
    verbose=1
)

grid_search.fit(X_naive, y)

resultados = grid_search.cv_results_
melhores_indices = np.argsort(resultados['mean_test_score'])[::-1][:2]

melhores_parametros = [resultados['params'][idx] for idx in melhores_indices]

print(f"Melhores parâmetros encontrados: {melhores_parametros}")


Fitting 10 folds for each of 4 candidates, totalling 40 fits
Melhores parâmetros encontrados: [{'priors': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'var_smoothing': 1e-07}, {'priors': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'var_smoothing': 1e-09}]


In [20]:
# Executando com as duas melhores combinações
for params in melhores_parametros:
  print(f"\nNaive Bayes para os hiperparâmetros: {params}\n")

  gnb = GaussianNB(
      var_smoothing=params['var_smoothing'],
      priors=params['priors']
    )

  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  accuracies_fold = cross_val_score(gnb, X_naive, y, cv=kf, scoring='accuracy')
  f1_scores_fold = cross_val_score(gnb, X_naive, y, cv=kf, scoring='f1_weighted')

  for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
      X_train, X_test = X_naive[train_index], X_naive[test_index]
      y_train, y_test = y[train_index], y[test_index]

      gnb.fit(X_train, y_train)
      y_pred = gnb.predict(X_test)

      # Acurácia e o F1-Score do fold atual
      print(f"Fold {fold}\nAcurácia = {accuracies_fold[fold-1]:.4f}\nF1-Score = {f1_scores_fold[fold-1]:.4f}\n")

  # Cálculo das médias e o desvio padrão
  media_acuracia = np.mean(accuracies_fold)
  desvio_acuracia = np.std(accuracies_fold)

  media_f1 = np.mean(f1_scores_fold)
  desvio_f1 = np.std(f1_scores_fold)

  print(f"Média de Acurácia: {media_acuracia:.4f}")
  print(f"Desvio Padrão de Acurácia: {desvio_acuracia:.4f}")
  print(f"Média de F1-Score: {media_f1:.4f}")
  print(f"Desvio Padrão de F1-Score: {desvio_f1:.4f}")


Naive Bayes para os hiperparâmetros: {'priors': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'var_smoothing': 1e-07}

Fold 1
Acurácia = 0.8000
F1-Score = 0.7802

Fold 2
Acurácia = 0.8000
F1-Score = 0.7944

Fold 3
Acurácia = 0.8000
F1-Score = 0.7802

Fold 4
Acurácia = 0.9333
F1-Score = 0.9327

Fold 5
Acurácia = 0.8667
F1-Score = 0.8649

Fold 6
Acurácia = 0.8000
F1-Score = 0.7908

Fold 7
Acurácia = 0.9000
F1-Score = 0.8995

Fold 8
Acurácia = 0.8333
F1-Score = 0.8318

Fold 9
Acurácia = 0.8667
F1-Score = 0.8649

Fold 10
Acurácia = 0.7667
F1-Score = 0.7532

Média de Acurácia: 0.8367
Desvio Padrão de Acurácia: 0.0504
Média de F1-Score: 0.8293
Desvio Padrão de F1-Score: 0.0561

Naive Bayes para os hiperparâmetros: {'priors': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'var_smoothing': 1e-09}

Fold 1
Acurácia = 0.8000
F1-Score = 0.7802

Fold 2
Acurácia = 0.8000
F1-Score = 0.7944

Fold 3
Acurácia = 0.8000
F1-Score = 0.7802

Fold 4
Acurácia = 0.9333
F1-Score =

# Regressão Logística

In [21]:
from sklearn.linear_model import LogisticRegression

y = np.array(classes)

grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    param_grid={
      'C': [0.1, 1, 10],  # Parâmetros de regularização
      'solver': ['lbfgs', 'liblinear']
    },
    scoring=make_scorer(f1_score, average='weighted'),
    cv=10,
    n_jobs=1,
    verbose=1
)

grid_search.fit(X, y)

resultados = grid_search.cv_results_
melhores_indices = np.argsort(resultados['mean_test_score'])[::-1][:2]

melhores_parametros = [resultados['params'][idx] for idx in melhores_indices]

print(f"Melhores parâmetros encontrados: {melhores_parametros}")



Fitting 10 folds for each of 6 candidates, totalling 60 fits
Melhores parâmetros encontrados: [{'C': 10, 'solver': 'liblinear'}, {'C': 10, 'solver': 'lbfgs'}]


In [22]:
# Executando com as duas melhores combinações
for params in melhores_parametros:
  print(f"\nRegressão Logística para os hiperparâmetros: {params}\n")

  logreg = LogisticRegression(
      random_state=42,
      max_iter=1000,
      C=params['C'],
      solver=params['solver']
    )

  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  accuracies_fold = cross_val_score(logreg, X, y, cv=kf, scoring='accuracy')
  f1_scores_fold = cross_val_score(logreg, X, y, cv=kf, scoring='f1_weighted')

  for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      logreg.fit(X_train, y_train)
      y_pred = logreg.predict(X_test)

      # Acurácia e o F1-Score do fold atual
      print(f"Fold {fold}\nAcurácia = {accuracies_fold[fold-1]:.4f}\nF1-Score = {f1_scores_fold[fold-1]:.4f}\n")

  # Cálculo das médias e o desvio padrão
  media_acuracia = np.mean(accuracies_fold)
  desvio_acuracia = np.std(accuracies_fold)

  media_f1 = np.mean(f1_scores_fold)
  desvio_f1 = np.std(f1_scores_fold)

  print(f"Média de Acurácia: {media_acuracia:.4f}")
  print(f"Desvio Padrão de Acurácia: {desvio_acuracia:.4f}")
  print(f"Média de F1-Score: {media_f1:.4f}")
  print(f"Desvio Padrão de F1-Score: {desvio_f1:.4f}")


Regressão Logística para os hiperparâmetros: {'C': 10, 'solver': 'liblinear'}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 1.0000
F1-Score = 1.0000

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 1.0000
F1-Score = 1.0000

Fold 6
Acurácia = 0.9667
F1-Score = 0.9666

Fold 7
Acurácia = 1.0000
F1-Score = 1.0000

Fold 8
Acurácia = 1.0000
F1-Score = 1.0000

Fold 9
Acurácia = 1.0000
F1-Score = 1.0000

Fold 10
Acurácia = 1.0000
F1-Score = 1.0000

Média de Acurácia: 0.9967
Desvio Padrão de Acurácia: 0.0100
Média de F1-Score: 0.9967
Desvio Padrão de F1-Score: 0.0100

Regressão Logística para os hiperparâmetros: {'C': 10, 'solver': 'lbfgs'}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 1.0000
F1-Score = 1.0000

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 1.0000
F1-Score = 1.0000

Fold 6
Acurácia = 0.9667
F1-Score = 0.9666

Fold 7
Acurácia = 1

# Rede Neural MLP

In [23]:
from sklearn.neural_network import MLPClassifier

y = np.array(classes)

grid_search = GridSearchCV(
    estimator=MLPClassifier(random_state=42,max_iter=200),
    param_grid={
      'hidden_layer_sizes': [(100,), (50, 50)],  # Diferentes n° de neurônios e camadas
      'learning_rate': ['constant', 'adaptive'],  # Tipos de taxa de aprendizado
    },
    scoring=make_scorer(f1_score, average='weighted'),
    cv=10,
    n_jobs=1,
    verbose=1
)

grid_search.fit(X, y)

resultados = grid_search.cv_results_
melhores_indices = np.argsort(resultados['mean_test_score'])[::-1][:2]

melhores_parametros = [resultados['params'][idx] for idx in melhores_indices]

print(f"Melhores parâmetros encontrados: {melhores_parametros}")



Fitting 10 folds for each of 4 candidates, totalling 40 fits
Melhores parâmetros encontrados: [{'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive'}, {'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant'}]


In [24]:
# Executando com as duas melhores combinações
for params in melhores_parametros:
  print(f"\nRede Neural para os hiperparâmetros: {params}\n")

  mlp = MLPClassifier(
    hidden_layer_sizes=params['hidden_layer_sizes'],
    learning_rate=params['learning_rate'],
    random_state=42,
    max_iter=200
  )

  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  accuracies_fold = cross_val_score(mlp, X, y, cv=kf, scoring='accuracy')
  f1_scores_fold = cross_val_score(mlp, X, y, cv=kf, scoring='f1_weighted')

  for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      mlp.fit(X_train, y_train)
      y_pred = mlp.predict(X_test)

      # Acurácia e o F1-Score do fold atual
      print(f"Fold {fold}\nAcurácia = {accuracies_fold[fold-1]:.4f}\nF1-Score = {f1_scores_fold[fold-1]:.4f}\n")

  # Cálculo das médias e o desvio padrão
  media_acuracia = np.mean(accuracies_fold)
  desvio_acuracia = np.std(accuracies_fold)

  media_f1 = np.mean(f1_scores_fold)
  desvio_f1 = np.std(f1_scores_fold)

  print(f"Média de Acurácia: {media_acuracia:.4f}")
  print(f"Desvio Padrão de Acurácia: {desvio_acuracia:.4f}")
  print(f"Média de F1-Score: {media_f1:.4f}")
  print(f"Desvio Padrão de F1-Score: {desvio_f1:.4f}")


Rede Neural para os hiperparâmetros: {'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive'}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 1.0000
F1-Score = 1.0000

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 0.9333
F1-Score = 0.9327

Fold 6
Acurácia = 1.0000
F1-Score = 1.0000

Fold 7
Acurácia = 1.0000
F1-Score = 1.0000

Fold 8
Acurácia = 1.0000
F1-Score = 1.0000

Fold 9
Acurácia = 1.0000
F1-Score = 1.0000

Fold 10
Acurácia = 1.0000
F1-Score = 1.0000

Média de Acurácia: 0.9933
Desvio Padrão de Acurácia: 0.0200
Média de F1-Score: 0.9933
Desvio Padrão de F1-Score: 0.0202

Rede Neural para os hiperparâmetros: {'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant'}

Fold 1
Acurácia = 1.0000
F1-Score = 1.0000

Fold 2
Acurácia = 1.0000
F1-Score = 1.0000

Fold 3
Acurácia = 1.0000
F1-Score = 1.0000

Fold 4
Acurácia = 1.0000
F1-Score = 1.0000

Fold 5
Acurácia = 0.9333
F1-Score = 0.9327

Fold 6
Acurácia =